In [4]:
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys,os

#Load the model

json_file = open("model-bw28.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("model-bw28.h5")
print("Loaded model from disk")

cap = cv2.VideoCapture(0)

# Category dictionary
categories = {0: 'Calm down', 1: 'Church', 2: 'Hello', 3: 'Help', 4: 'I love you', 5: 'No', 6: 'Nothing', 7: 'Pray', 8: 'Stand', 9: 'Where', 10: 'Why', 11: 'Yes'}

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    # Got this from collect-data.py
    # Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    
    # Resizing the ROI so it can be fed to the model for prediction
    roi = cv2.resize(roi, (64, 64)) 
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    # Batch of 1
    result = loaded_model.predict(test_image.reshape(1, 64, 64, 1))
    prediction = {'Calm down': result[0][0],
                  'Church': result[0][1],
                  'Hello': result[0][2],
                  'Help': result[0][3],
                  'I love you': result[0][4],
                  'No': result[0][5],
                  'Nothing': result[0][6],
                  'Pray': result[0][7],
                  'Stand': result[0][8],
                  'Where': result[0][9],
                  'Why': result[0][10],
                  'Yes': result[0][11]
                  }
    # Sorting based on top prediction
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    
    # Displaying the predictions
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)    
    cv2.imshow("Frame", frame)
    
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
        
 
cap.release()
cv2.destroyAllWindows()

Loaded model from disk
